Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

e:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [17]:
graph = tf.Graph()
with graph.as_default():
...
 loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ \
    tf.scalar_mul(beta, tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
    


Initialized
Minibatch loss at step 0: 16.563961
Minibatch accuracy: 7.0%
Validation accuracy: 13.7%
Minibatch loss at step 500: 2.034568
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 2.051915
Minibatch accuracy: 69.5%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 1.154994
Minibatch accuracy: 78.1%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 1.070973
Minibatch accuracy: 76.6%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 1.365877
Minibatch accuracy: 76.6%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.969469
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
#############################
Test accuracy: 85.7% with beta=0.000010


##### summary
With
```python
batch_size = 128
num_hidden_nodes = 1024
beta = 1e-3
num_steps = 3001
```

Results
* Test accuracy: 88.5% with beta=0.000000 (no L2 regulization)
* Test accuracy: 86.7% with beta=0.000010
* Test accuracy: 88.8% with beta=0.000100
* **Test accuracy: 92.6% with beta=0.001000**
* Test accuracy: 89.7% with beta=0.010000
* Test accuracy: 82.2% with beta=0.100000
* Test accuracy: 10.0% with beta=1.000000


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
batch_size = 128
num_steps = 3001
beta = 0.001

################################################
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = 1 #### only change in problem 2
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
    #if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print('#############################')
  print("Test accuracy: %.1f%% with beta=%f" % (accuracy(test_prediction.eval(), test_labels), beta))

Initialized
Minibatch loss at step 0: 20.999857
Minibatch accuracy: 4.7%
Validation accuracy: 8.8%
Minibatch loss at step 500: 1.807533
Minibatch accuracy: 100.0%
Validation accuracy: 54.6%
Minibatch loss at step 1000: 1.104561
Minibatch accuracy: 100.0%
Validation accuracy: 58.0%
Minibatch loss at step 1500: 0.679056
Minibatch accuracy: 100.0%
Validation accuracy: 60.6%
Minibatch loss at step 2000: 0.421669
Minibatch accuracy: 100.0%
Validation accuracy: 62.7%
Minibatch loss at step 2500: 0.266144
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 3000: 0.172263
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
#############################
Test accuracy: 71.4% with beta=0.001000


With 
```python
batch_size = 128
num_steps = 3001
beta = 0.001
```
* Problem 1: Test accuracy: 88.0% with beta=0.001000
* Problem 2: Test accuracy: 71.4% with beta=0.001000

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 128
num_steps = 30001
beta = 0.001
keep_rate = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  logits = tf.nn.dropout(logits, keep_rate)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
    tf.scalar_mul(beta, tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

################################################
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
    #if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print('#############################')
  print("Test accuracy: %.1f%% with beta=%f" % (accuracy(test_prediction.eval(), test_labels), beta))

Initialized
Minibatch loss at step 0: 30.585913
Minibatch accuracy: 8.6%
Validation accuracy: 13.1%
Minibatch loss at step 500: 3.897573
Minibatch accuracy: 39.1%
Validation accuracy: 75.6%
Minibatch loss at step 1000: 2.733506
Minibatch accuracy: 46.9%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 1.750470
Minibatch accuracy: 53.1%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 1.822119
Minibatch accuracy: 43.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 1.804763
Minibatch accuracy: 39.8%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 1.631987
Minibatch accuracy: 41.4%
Validation accuracy: 78.5%
Minibatch loss at step 3500: 1.524636
Minibatch accuracy: 52.3%
Validation accuracy: 78.4%
Minibatch loss at step 4000: 1.485615
Minibatch accuracy: 44.5%
Validation accuracy: 78.7%
Minibatch loss at step 4500: 1.581459
Minibatch accuracy: 48.4%
Validation accuracy: 79.5%
Minibatch loss at step 5000: 1.412006
Minibatch accuracy: 46.9%
Validation accurac

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
